# Kaggle Environment & Numpy 2.0 Compatibility Test
This notebook audits the pre-installed libraries on Kaggle and tests their compatibility with the new Numpy 2.0 ABI.
Run this to verify if the "Synchronized Stack Upgrade" is necessary.

In [ ]:
# 1. Audit Initial Environment
import sys
import platform
import os
import importlib
import subprocess

print(f"Python: {sys.version.split()[0]}")
print(f"Platform: {platform.platform()}")

libs = ['numpy', 'torch', 'matplotlib', 'scipy', 'pandas', 'ctranslate2']
print("\n--- Pre-installed Versions ---")
for lib in libs:
    try:
        mod = importlib.import_module(lib)
        print(f"{lib:<12}: {mod.__version__}")
        if lib == 'numpy':
            print(f"  Location    : {os.path.dirname(mod.__file__)}")
    except ImportError:
        print(f"{lib:<12}: NOT INSTALLED")

# Check CUDA environment
print("\n--- CUDA Environment ---")
print(f"LD_LIBRARY_PATH: {os.environ.get('LD_LIBRARY_PATH', 'Not Set')}")
cuda_check = subprocess.run("ls -d /usr/local/cuda* 2>/dev/null", shell=True, capture_output=True, text=True)
print(f"System CUDA:\n{cuda_check.stdout.strip()}")

In [ ]:
# 2. Check Numpy Compatibility of System Torch
# Inspect linking of torch C extensions to see what they expect
import torch
import sys

print(f"Torch Version: {torch.__version__}")
print(f"Torch File: {torch.__file__}")

try:
    # Try to find linkage info (Linux specific)
    site_packages = os.path.dirname(os.path.dirname(torch.__file__))
    libs_path = os.path.join(os.path.dirname(torch.__file__), 'lib')
    # Run ldd on a torch shared object
    so_file = os.path.join(os.path.dirname(torch.__file__), '_C.so') # Older torch
    if not os.path.exists(so_file):
        # Newer torch structure
        import glob
        so_files = glob.glob(os.path.join(os.path.dirname(torch.__file__), '*.so'))
        if so_files: so_file = so_files[0]
    
    if os.path.exists(so_file):
        print(f"\nChecking dependencies for: {os.path.basename(so_file)}")
        ldd_res = subprocess.run(f"ldd {so_file} | grep numpy", shell=True, capture_output=True, text=True)
        print(f"LDD Result (Direct Link): {ldd_res.stdout.strip() or 'No direct link found (likely statically linked or via Python API)'}")
    else:
        print("Could not find .so file to check linkage.")
except Exception as e:
    print(f"Linkage check failed: {e}")

In [ ]:
# 3. The "Crash Test": Install Numpy 2.0 ONLY
# This simulates what happens if we only upgrade numpy (required by WhisperJAV) 
# but try to keep system Torch/Matplotlib.
print("Installing Numpy >= 2.0...")
!pip install -q "numpy>=2.0.0"

In [ ]:
# 4. Verify System Breakage
# We run this in a subprocess to ensure clean import state
import sys
import subprocess

test_script = """
import sys
try:
    import numpy as np
    print(f"Loaded Numpy: {np.__version__}")
except ImportError as e:
    print(f"CRITICAL: Numpy failed to load: {e}")
    sys.exit(1)

print("-" * 20)

# Test Torch
try:
    import torch
    print(f"SUCCESS: Torch {torch.__version__} imports with Numpy 2.0")
    # Try a simple tensor operation
    x = torch.tensor([1.0, 2.0])
    print(f"Torch Tensor Op: OK")
except Exception as e:
    print(f"FAILURE: Torch crashed/failed with Numpy 2.0: {e}")

print("-" * 20)

# Test Matplotlib
try:
    import matplotlib.pyplot as plt
    print(f"SUCCESS: Matplotlib {plt.matplotlib.__version__} imports with Numpy 2.0")
except Exception as e:
    print(f"FAILURE: Matplotlib crashed/failed with Numpy 2.0: {e}")
"""

print("Running import tests in subprocess...")
res = subprocess.run([sys.executable, "-c", test_script], capture_output=True, text=True)
print(res.stdout)
if res.stderr:
    print("ERRORS:")
    print(res.stderr)